<a href="https://colab.research.google.com/github/kimdinhloc/Applied_Parallel_Programming_HK2_2021_2022/blob/main/Report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **ĐỒ ÁN LẬP TRÌNH SONG SONG ỨNG DỤNG**

# BACKGROUND REMOVAL WITH GRABCUT AND SUPERPIXELS

## Giáo viên: Trần Trung Kiên

### Sinh viên:
- 1712568	KIM ĐÌNH LỘC	
- 1712787	NGUYỄN VĂN THÌN	
- 1712898	TRẦN VIỆT VĂN	


#0. Ý NGHĨA TỪ NGỮ

**GrabCut** là phương pháp để phân đoạn một cách chính xác tiền cảnh của hình ảnh khỏi hậu cảnh.

**Superpixel** là kết quả của việc nhóm các pixel theo cảm nhận màu sắc, hoặc theo cách nhìn khác, đó là kết quả của sự phân đoạn hình ảnh.([Tu-chemnitz.de. 2022. Superpixel | TU Chemnitz](https://www.tu-chemnitz.de/etit/proaut/en/research/superpixel.html))


**Superpixel segmentation** là quá trình phân đoạn hình ảnh thành nhiều superpixels.

 <img src="https://raw.githubusercontent.com/kimdinhloc/Applied_Parallel_Programming_HK2_2021_2022/main/Illustrated%20image/Dog_SLIC.jpg" 
     width="640" 
     height="455" />



#1. MÔ TẢ ỨNG DỤNG

**Ứng dụng mà nhóm chọn là BACKGROUND REMOVAL WITH GRABCUT AND SUPERPIXELS**.  

Lý do chọn đề tài:

<img src="https://raw.githubusercontent.com/kimdinhloc/Applied_Parallel_Programming_HK2_2021_2022/main/Illustrated%20image/compair.jpg.png" 
     width="1218" 
     height="300" />

**Input:**  
Một tấm ảnh RGB, mô hình yolov3
  
  <img src="https://raw.githubusercontent.com/kimdinhloc/Applied_Parallel_Programming_HK2_2021_2022/main/Illustrated%20image/Dog.JPG" 
     width="640" 
     height="455" />

**Output**  
Ảnh đầu vào được remove background.  
<img src="https://raw.githubusercontent.com/kimdinhloc/Applied_Parallel_Programming_HK2_2021_2022/main/Illustrated%20image/Dog_output.jpg" 
     width="640" 
     height="455" />


- Ý nghĩa của ứng dụng:
  - Hỗ trợ tách/lọc nền trong việc chỉnh sửa ảnh.
  - Nghiên cứu về segmentation việc xoá background.
  - Giảmđộ phức tạp của các bước xử lý ảnh tiếp theo.
- Lý do cần tăng tốc ứng dụng:
	- Ứng dụng sẽ chạy chậm nếu cài đặt tuần tự.
	- Nhu cầu xử lý dữ liệu lớn, ảnh có độ phân giải cao hay việc xử lý theo thời gian thực
- Tiềm năng song song hóa của ứng dụng: 
  - Song song hoá việc chuyển từ hệ màu RGB sang hệ màu CELAB
  - Thực hiện đồng thời quá trình khởi tạo các cụm của SLIC.
  - Thực hiện đồng thời quá trình tính từ trung tâm cụm đến từng pixel trong vùng.
  - Thực hiện song song hoá quá trình xử lý hình ảnh
- Thách thức:
  - Quá trình song song hoá việc kết nối các pixel trong SLIC có thể là một việc khó khăn.
  - Việc song song hoá mô hình GRABCUT của OpenCV rất khó khăn.
  - Thời gian xử lý lâu hơn so với model Machine Learning/deep learning.
  - Độ chính xác không cao so với deep learning.

#2. QUÁ TRÌNH THỰC HIỆN ĐỒ ÁN

<img src="https://raw.githubusercontent.com/kimdinhloc/Applied_Parallel_Programming_HK2_2021_2022/main/Illustrated%20image/Model/timeline.JPG" 
     width="1152" 
     height="360" />

#3. CÀI ĐẶT TUẦN TỰ

##3.1. Mô hình tuần tự, không jit

###3.1.1. Thiết kế

<img src="https://raw.githubusercontent.com/kimdinhloc/Applied_Parallel_Programming_HK2_2021_2022/main/Illustrated%20image/Model/non_jit.JPG" 
     width="1152" 
     height="360" />

### 3.1.2. Thử nghiệm

Xem chi tiết quá trình thực nghiệm tại [link colab](https://colab.research.google.com/drive/1Uf6rruTo7wDgZ8h47HaaxB5ip1xA8_IR?usp=sharing) tại mục 2. SO SÁNH HIỆU SUẤT TỪNG GIỮA CÁC MÔ HÌNH 

Thời gian thực hiện là 3min 49s

##3.2. Mô hình tuần tự, jit lần 1

Dựa trên mô hình tuần tự chưa jit, nhóm tiến hành jit các hàm có thể được jit

###3.2.1. Thiết kế

<img src="https://raw.githubusercontent.com/kimdinhloc/Applied_Parallel_Programming_HK2_2021_2022/main/Illustrated%20image/Model/jit_v1.JPG" 
     width="1152" 
     height="360" />

###3.2.2. Thực nghiệm

Xem chi tiết quá trình thực nghiệm tại [link colab](https://colab.research.google.com/drive/1Uf6rruTo7wDgZ8h47HaaxB5ip1xA8_IR?usp=sharing) tại mục 2. SO SÁNH HIỆU SUẤT TỪNG GIỮA CÁC MÔ HÌNH 

Thời gian thực hiện là 3min 8s

Thời gian của @jit giảm khoảng từ 20% so với không sử dụng jit.

##3.3. Mô hình tuần tự, jit lần 2

Mô hình tuần tự và jit lần thứ 2 dựa trên kinh nghiệm của việc jit lần 1, sự khó khăn của code khi song song quá lần 1 gặp nhiều vướng mắc, khó khăn.  
Từ đó, nhóm quyết định code mô hình tuần tự, jit lần 2 để giải quyết các khó khăn đồng thời tạo nền tảng tốt hơn đến việc song song hoá lần 2

###3.3.1. Thiết kế

<img src="https://raw.githubusercontent.com/kimdinhloc/Applied_Parallel_Programming_HK2_2021_2022/main/Illustrated%20image/Model/jit_v2_1.JPG" 
     width="1152" 
     height="360" />

<img src="https://raw.githubusercontent.com/kimdinhloc/Applied_Parallel_Programming_HK2_2021_2022/main/Illustrated%20image/Model/jit_v2_2.JPG" 
     width="1152" 
     height="360" />

###3.3.2. Thực nghiiệm

Xem chi tiết quá trình thực nghiệm tại [link colab](https://colab.research.google.com/drive/1Uf6rruTo7wDgZ8h47HaaxB5ip1xA8_IR?usp=sharing) tại mục 2. SO SÁNH HIỆU SUẤT TỪNG GIỮA CÁC MÔ HÌNH 

Thời gian thực hiện 1min 12s

Việc tối ưu cũng như jit lần 2 khiến cho mô hình giảm thời gian rất nhiều so với lần 1 và chưa jit, cụ thể:
- Giảm thời gian chỉ còn 38.29% so với jit lần 1 (Tức là nhanh hơn 2.61 lần so với jit lần 1)
- Giảm thời gian chỉ còn 32.43% so với chưa jit (Tức là nhanh hơn 3.08 lần so với chưa jit)

#4. CÀI ĐẶT SONG SONG

##4.1. Song song hoá lần 1

###4.1.1. Thiết kế

Mô hình được thiết kế theo sự kế thừa của mô hình tuần tự jit lần 1

<img src="https://raw.githubusercontent.com/kimdinhloc/Applied_Parallel_Programming_HK2_2021_2022/main/Illustrated%20image/Model/parallel_v1.JPG" 
     width="1152" 
     height="360" />

###4.1.2. Thực nghiệm

Xem chi tiết quá trình thực nghiệm tại [link colab](https://colab.research.google.com/drive/1Uf6rruTo7wDgZ8h47HaaxB5ip1xA8_IR?usp=sharing) tại mục 2. SO SÁNH HIỆU SUẤT TỪNG GIỮA CÁC MÔ HÌNH 

Thời gian thực nghiệm 3min 7s

Thời gian @cuda.jit giảm 48 giây so với không sử dụng jit.
Thời gian @cuda.jit giảm 1 giây so với jit lần 1.

Lý do: Một số phần chưa thể cuda.jit do thực hiện tính toán và tạo mảng trong hàm khiến cho cuda.jit không thể tạo kernel.  
Hướng giải quyết: Thực hiện tái cấu trúc code phần Superpixel

##4.2.Song song hóa lần 2

###4.2.1. **Thiết kế dự kiến**

<img src="https://raw.githubusercontent.com/kimdinhloc/Applied_Parallel_Programming_HK2_2021_2022/main/Illustrated%20image/Model/parallel_v2.JPG" 
     width="1152" 
     height="360" />

###4.2.2. **Thực nghiệm dự kiến**

BUG ĐÃ XÂM CHIẾM KHU VỰC NÀY :))))

#5. SAI SỐ MÔ HÌNH

Xem chi tiết quá trình thực nghiệm tại [link colab](https://colab.research.google.com/drive/1Uf6rruTo7wDgZ8h47HaaxB5ip1xA8_IR?usp=sharing) tại mục 3. SO SÁNH SAI SỐ

BUG CŨNG ĐÃ XÂM CHIẾM KHU VỰC NÀY :))))

#6. TỔNG KẾT THỰC NGHIỆM

<table>
<thead>
<tr>
<th>Image</th>
<th>Non jit</th>
<th>jit version 1</th>
<th>jit version 2</th>
<th>parallel version 1</th>
<th>parallel version 2</th>
</tr>
</thead>
<tbody>
<tr>
<td>600x455</td>
<td>228</td>
<td>188</td>
<td>72</td>
<td>187</td>
<td>none</td>
</tr>
<tr>
<td>3264x2488</td>
<td>>1500</td>
<td>none</td>
<td>none</td>
<td>none</td>
<td>none</td>
</tr>
</tbody>
</table>

#7. KINH NGIỆM TÍCH LUỸ TỪ ĐỒ ÁN

Mỗi thành viên trong nhóm đã học được những gì từ đồ án

**1712568 - KIM ĐÌNH LỘC**

abc

**1712787 - NGUYỄN VĂN THÌN**

abc

**1712898 - TRẦN VIỆT VĂN**

abc

#TÀI LIỆU HÌNH ẢNH

WIKIPEDIA - https://upload.wikimedia.org/wikipedia/commons/0/05/Cow-bw.JPG  
REDDIT - https://www.reddit.com/r/dogpictures/comments/6puk20/i_am_speed_because_i_am_speedy/

# TÀI LIỆU THAM KHẢO

Grabcut https://cvg.ethz.ch/teaching/cvl/2012/grabcut-siggraph04.pdf

Achanta, Radhakrishna, et al. Slic superpixels. No. REP_WORK. 2010. [Chi tiết tại đây!](https://infoscience.epfl.ch/record/149300)

Le, Cuong Vo, et al. "Superpixel-based background removal for accuracy salience person re-identification." 2016 IEEE International Conference on Consumer Electronics-Asia (ICCE-Asia). IEEE, 2016. [Chi tiết tại đây!](https://ieeexplore.ieee.org/abstract/document/7804806/)

